# Введение в Rerank с Compressa

Современные системы информационного поиска постоянно совершенствуются, стремясь предоставить пользователям наиболее релевантные результаты. Одним из эффективных методов улучшения качества поиска является использование двухэтапного подхода с применением переранжирования.

Платформа Compressa предоставляет Rerank модель для реализации такого подхода.

Процесс работы может выглядеть следующим образом:

1. Сначала система берет вопрос пользователя и извлекает топ-100 документов из датасета, используя либо лексический, либо семантический поиск.
2. Затем эти 100 документов вместе с исходным вопросом передаются в Compressa Rerank.
3. Для каждого документа вычисляется оценка релевантности.
4. На основе полученных оценок документы переранжируются, предоставляя пользователю наиболее подходящие результаты.

Особенно эффективным этот метод может быть в случаях, когда отсутствуют обучающие данные для настройки поисковой системы.

В этом руководстве мы рассмотрим, как использовать Compressa Rerank для улучшения результатов поиска, и продемонстрируем её применение на практических примерах.

In [19]:
# Установка необходимых библиотек
# !pip install git+https://github.com/insight-stream/langchain_compressa.git
!pip install langchain


# Импорт необходимых библиотек
import os
import requests
import numpy as np
from time import time
from typing import List
from pprint import pprint
from langchain_compressa import CompressaRerank
from langchain.schema import Document

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 5.1 MB/s eta 0:00:0000:0100:01m


Для ячейки ниже вам понадобится API ключ Compressa. Вы можете получить его после [регистрации](compressa.ai/form). 

In [2]:
os.environ["COMPRESSA_API_KEY"] = "ваш_ключ_здесь"

In [20]:
# Инициализация Compressa Rerank
reranker = CompressaRerank()

# Пример запроса и "документов"
query = "Кто является автором картины 'Черный квадрат'?"
docs = [
    "Казимир Малевич создал 'Черный квадрат' в 1915 году, что стало революционным событием в мире искусства.",
    "Илья Репин известен своими реалистичными картинами, такими как 'Бурлаки на Волге' и 'Иван Грозный и сын его Иван'.",
    "'Черный квадрат' Малевича считается одним из самых влиятельных произведений абстрактного искусства XX века.",
    "Василий Кандинский, современник Малевича, также работал в жанре абстрактного искусства.",
    "Малевич написал несколько версий 'Черного квадрата' между 1915 и 1930 годами.",
    "Картина 'Черный квадрат' впервые была выставлена на футуристической выставке '0,10' в Петрограде.",
    "Помимо 'Черного квадрата', Малевич известен своими работами в стиле супрематизма.",
    "Марк Шагал, еще один известный русский художник, работал в стиле модернизма и сюрреализма.",
    "В Третьяковской галерее в Москве хранится одна из версий 'Черного квадрата' Малевича.",
    "Картина 'Черный квадрат' вызвала множество дискуссий и интерпретаций в мире искусства."
]

# Создаем объекты типа Document из наших вопросов
doc_objects = [
    Document(page_content=doc, metadata={"index": idx})
    for idx, doc in enumerate(docs)
]

## Используем Compressa Rerank

В следующей ячейке мы вызовем Compressa Rerank для ранжирования `docs` на основе их релевантности запросу `query`

In [23]:
results = reranker.compress_documents(query=query, documents=doc_objects)

top_n = 3 # Измените top_n, чтобы изменить количество возвращаемых результатов.
sorted_results = sorted(results, key=lambda x: x.metadata['relevance_score'], reverse=True)
selected_results = sorted_results[:top_n]

print(query)

for idx, r in enumerate(selected_results):
    doc_index = r.metadata["index"]
    doc_text = r.page_content if doc_index < len(docs) else "Текст не найден"
    print(f"Ранг документа: {idx + 1}, Индекс документа: {doc_index}")
    print(f"Документ: {doc_text}")
    print(f"Оценка релевантности: {r.metadata['relevance_score']:.2f}")
    print("\n")

Кто является автором картины 'Черный квадрат'?
Ранг документа: 1, Индекс документа: 0
Документ: Казимир Малевич создал 'Черный квадрат' в 1915 году, что стало революционным событием в мире искусства.
Оценка релевантности: 0.96


Ранг документа: 2, Индекс документа: 2
Документ: 'Черный квадрат' Малевича считается одним из самых влиятельных произведений абстрактного искусства XX века.
Оценка релевантности: 0.89


Ранг документа: 3, Индекс документа: 4
Документ: Малевич написал несколько версий 'Черного квадрата' между 1915 и 1930 годами.
Оценка релевантности: 0.74




## Поиск по Википедии - End2end демо

В следующем примере мы покажем, как использовать Rerank модель для улучшения поиска по статьям русскоязычной Wikipedia. Для создания удобного датасета мы уменьшили mrtydi-v1.1-russian до порядка 500к отрывков из статей, сохранив "правильные" ответы для всех вопросов, которые будем использовать ниже, и добавили другие рандомные статьи. 

Мы будем использовать лексический поиск BM25 для извлечения топ-100 отрывков, соответствующих запросу, а затем отправим эти 100 отрывков и запрос в Compressa Rerank, чтобы получить отранжированные результаты. 

Мы выведем топ-3 результата согласно BM25 (как это используется, например, в Elasticsearch) и улучшенный результат от нашей модели Compressa Rerank.

Для начала установим и импортируем дополнительные библиотеки.

In [14]:
!pip install -U  rank_bm25
!pip install gdown

In [15]:
import gdown
import json
import gzip
import os
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm

/var/folders/bv/dtp_xcrd7sz0j1r72rzt1wb80000gn/T/ipykernel_41506/3115818506.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [16]:
#Скачиваем датасет c google диска Сompressa, он уже нарезан на отрывки
file_id = '1u-3EQ4yoYys1wSSK_EifIPPRyh_6oaR6'
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, 'mrtydi-russian-subset.jsonl', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u-3EQ4yoYys1wSSK_EifIPPRyh_6oaR6
From (redirected): https://drive.google.com/uc?id=1u-3EQ4yoYys1wSSK_EifIPPRyh_6oaR6&confirm=t&uuid=11d10214-b3d6-4337-9bb7-490b2ae615bd
To: /Users/valminsky/mrtydi-russian-subset.jsonl
100%|████████████████████████████████████████| 368M/368M [02:01<00:00, 3.03MB/s]


'mrtydi-russian-subset.jsonl'

In [24]:
with open('mrtydi-russian-subset.jsonl', 'r', encoding='utf-8') as f:
    passages = [json.loads(line) for line in tqdm(f)]

print(f"Всего отрывков в подмножестве: {len(passages)}")
print(f"Пример первого отрывка:")
print(f"ID: {passages[0]['id']}")
print(f"Содержание: {passages[0]['contents'][:300]}...")  # Показываем первые 200 символов содержания

0it [00:00, ?it/s]

Всего отрывков в подмножестве: 524906
Пример первого отрывка:
ID: 2725347#0
Содержание: Климат Астрахани

Климат полупустынный, засушливый, тёплый, формируется под воздействием циркуляционных атмосферных процессов южной зоны умеренных широт. Территория доступна также выносу арктических, тропических (из Средиземноморья и Ирана), а также морских (с Атлантики) и континентальных (из Казахс...


In [25]:
# Мы будем сравнивать результаты с обычным лексическим поиском (поиском по ключевым словам). 
# Здесь мы используем алгоритм BM25, который реализован в пакете rank_bm25.
# Мы приводим наш текст к нижнему регистру и удаляем стоп-слова при индексации

# Для начала зададим список русских стоп-слов, чтобы улучшить процесс токенизации. Список можно расширить или взять готовый, он приведен для примера.
RUSSIAN_STOP_WORDS = set([
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так',
    'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было',
    'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг',
    'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж',
    'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть',
    'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего',
    'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого',
    'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас',
    'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой',
    'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая',
    'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'
])

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)
        if len(token) > 0 and token not in RUSSIAN_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage['contents']))

bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/524906 [00:00<?, ?it/s]

In [26]:
# Эта функция будет искать во всех отрывках из Википедии те, которые отвечают на запрос. 
# Затем мы выполняем переранжирование, используя Compressa Rerank

def search(query, top_k=3, num_candidates=100):
    print("Наш вопрос:", query)

    # Поиск BM25, то есть поиск по ключевым словам 
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print(f"Топ-3 результата поиска BM25 по ключевым словам")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]['contents'].replace("\n", " ")))

    # Превратим отрывки в документы для переранжирования
    docs = [
        Document(page_content=passages[hit['corpus_id']]['contents'], metadata={"corpus_id": hit['corpus_id']})
        for hit in bm25_hits
    ]
    
    # Реранжируем документы
    reranked_results = reranker.compress_documents(query=query, documents=docs)
    sorted_results = sorted(reranked_results, key=lambda x: x.metadata["relevance_score"], reverse=True)

    print(f"\nТоп-3 результата после переранжирования ({len(bm25_hits)} результатов BM25 переранжированы)")
    for hit in sorted_results[:top_k]:
        print("\t{:.3f}\t{}".format(hit.metadata['relevance_score'], hit.page_content.replace("\n", " ")))

In [27]:
search(query = "В каком году была создана компания Apple?")

Наш вопрос: В каком году была создана компания Apple?
Топ-3 результата поиска BM25 по ключевым словам
	21.278	Kowa company  После Второй мировой войны компания разнообразила свою деятельность. В 1946 году была создана компания , которая начала производство оптического оборудования. В 1947 году компания начала производство лекарств и в 1954 году была создана компания .
	21.001	ARM (компания)  Компания Advanced RISC Machines изначально была основана как совместное предприятие между Acorn Computers, Apple Computer (сегодня — Apple Inc.) и VLSI Technology.
	20.896	Apple  В 2001 году компания представила аудиоплеер iPod, быстро приобретший популярность.

Топ-3 результата после переранжирования (100 результатов BM25 переранжированы)
	0.999	Apple  Компания основана в Калифорнии Стивом Джобсом, Рональдом Уэйном и Стивом Возняком, собравшими в середине 1970-х годов свой первый персональный компьютер на базе процессора «MOS Technology 6502». Продав несколько десятков таких компьютеров, молодые п

In [28]:
search(query = "Где были найдены останки человека флоресского в 2003 году?")

Наш вопрос: Где были найдены останки человека флоресского в 2003 году?
Топ-3 результата поиска BM25 по ключевым словам
	26.472	Кабве  В 1921 году в одном из рудников в районе Брокен-Хилл были найдены останки ископаемого человека, названного «брокен-хиллский человек», или «родезийский человек».
	25.239	Форты и оборонительная линия Брестской крепости  В 2003 году в районе форта были найдены останки более тысячи воинов Первой мировой, погибших, видимо, в боях при отступлении русских войск в 1915 году.
	21.636	Реабилитация царской семьи  Останки пяти членов императорской семьи, а также их слуг, были найдены в июле 1991 года и идентифицированы. 17 июля 1998 года останки членов императорской семьи были захоронены в Петропавловском соборе Санкт-Петербурга. В июле 2007 года были найдены останки царевича Алексея и великой княжны Марии.

Топ-3 результата после переранжирования (100 результатов BM25 переранжированы)
	0.977	Человек флоресский  Останки человека флоресского были впервые обнаружены в

In [29]:
search(query = "Когда началось Возрождение?")

Наш вопрос: Когда началось Возрождение?
Топ-3 результата поиска BM25 по ключевым словам
	19.473	Высокогорская Успенско-Николаевская Чуркинская пустынь  Весной 2004 года началось возрождение монастыря.
	19.142	Неправославные храмы Ленинградской области  В конце 1980-х годов началось возрождение Церкви Ингрии.
	18.823	Ламаркизм  Но, тем не менее, возрождение ламаркизма началось именно с появлением эволюционной теории Дарвина в 1859 году.

Топ-3 результата после переранжирования (100 результатов BM25 переранжированы)
	0.992	Возрождение  Возрождение возникло в Италии, где первые его признаки были заметны ещё в XIII и XIV веках (в деятельности семейства Тони Парамони, Пизано, Джотто, Орканья и других), но оно твёрдо установилось только с 1420-х годов. Во Франции, Германии и других странах это движение началось значительно позже. К концу XV века оно достигло своего наивысшего расцвета. В XVI веке назревает кризис идей Возрождения, следствием чего является возникновение маньеризма и барокко.


In [30]:
search(query = "Сколько людей живет в Мехико в 2019 году?")

Наш вопрос: Сколько людей живет в Мехико в 2019 году?
Топ-3 результата поиска BM25 по ключевым словам
	15.948	Палочки для еды  Палочками пользуются 30 % людей — столько же, сколько пользуются вилкой. Прочие едят руками.
	15.030	Кипу  Для передачи экономико-статистических данных кипу использовали двойную запись[14], а при передаче сведений о производстве тех или иных продуктов труда учитывали не только фактическую, но и наличную и потенциальную производительность труда. Вот что пишет известный исследователь андских цивилизаций В. А. Кузьмищев: Кипу «знали», сколько человек проживало в любом из селений и во всём царстве, сколько из них было мужского и женского пола, как они были разбиты по возрасту и по состоянию здоровья, сколько среди них было женатых и вдовых, сколько ушло на войну и на общественные работы, сколько людей и какой работой занимались сегодня и сколько они могли произвести того или иного продукта и так далее и тому подобное. Но не только люди и результаты их труда, а сама

In [31]:
search(query="Какой язык является самым древним?")

Наш вопрос: Какой язык является самым древним?
Топ-3 результата поиска BM25 по ключевым словам
	23.911	Амальгамация  Этот метод был известен в античности, является самым древним из существующих методов очистки золота.
	16.787	Русский язык  Ру́сский язы́к ([ˈruskʲɪi̯ jɪˈzɨk])[~ 1] — один из восточнославянских языков, национальный язык русского народа. Является одним из наиболее распространённых языков мира — шестым среди всех языков мира по общей численности говорящих и восьмым по численности владеющих им как родным[1]. Русский является также самым распространённым славянским языком[2] и самым распространённым языком в Европе — географически и по числу носителей языка как родного[3].
	15.335	Махзор  Самым древним среди европейских махзоров считается «M.-Romania», или «Chasania schel Romania», или «Тригос» (то есть греков); известный библиограф Цеднер полагает, что впервые он был напечатан в Венеции в типографии Даниила Бомберга, а затем и в Константинополе (1573—76).

Топ-3 результата п

In [32]:
search(query="Какой климат в Бурятии?")

Наш вопрос: Какой климат в Бурятии?
Топ-3 результата поиска BM25 по ключевым словам
	15.889	Бурятия  Новая Бурятия, Информ-Полис, Молодёжь Бурятии, Московский комсомолец в Бурятии, Правда Бурятии, Буряад үнэн, Пятница, Центральная газета.
	14.186	Бурятия  Конституция Республики Бурятия Список полных кавалеров Ордена Славы из Бурятии Список Героев Российской Федерации из Бурятии Тункинский эксперимент
	13.638	Бурятия  Музей истории Бурятии Этнографический музей народов Забайкалья, Музей природы Бурятии, Художественный музей им. Сампилова, Музей истории города Улан-Удэ, Музей декабристов в Новоселенгинске.

Топ-3 результата после переранжирования (100 результатов BM25 переранжированы)
	0.996	Бурятия  Климат Бурятии резко континентальный. Зима холодная, с сухим морозом. В ноябре-декабре выпадают основные снегопады. Вторая половина зимы характеризуется малым количеством снега. Весна ветреная, с господствующими северо-западными ветрами, с заморозками и почти без осадков. Лето короткое, с жа

In [39]:
search(query="На какой реке расположен Тольятти?")

Наш вопрос: На какой реке расположен Тольятти?
Топ-3 результата поиска BM25 по ключевым словам
	19.356	Тольятти  Административный центр Ставропольского района Самарской области расположен в городе Тольятти, однако сам город в состав этого района не входит.
	17.637	Тольятти  Международный аэропорт Курумоч расположен в районе посёлка Берёза и находится в 70 км от центральной части Тольятти и в 35 км от границы областного центра Самары по автодороге М-5 «Урал».
	17.404	Микулинцы  Расположен на реке Серет.

Топ-3 результата после переранжирования (100 результатов BM25 переранжированы)
	0.860	Тольятти  Расположен на левом берегу Волги напротив Жигулей. Население: чел. (), 707 408[3] чел. (2018); самый крупный город России, не являющийся центром субъекта федерации.
	0.612	Тольятти  Город расположен непосредственно на границе трёх физико-географических районов: Самарской Луки, Мелекесского низменного Заволжья и лесостепного Заволжья — весьма различных между собой по рельефу, флоре, фауне, вед